In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

In [3]:
df=pd.read_csv('/home/siva/Downloads/kindle_reviews.csv')
df

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200
...,...,...,...,...,...,...,...,...,...,...
982614,982614,B00M13FNSS,"[2, 2]",5,Yasss hunny! This is a great read. That Dre is...,"07 23, 2014",A2Y66HD4J5S7QZ,Candi,A Hot Read Indeed!!,1406073600
982615,982615,B00M13FNSS,"[0, 0]",5,I ENJOYED THIS BOOK FROM BEGINNING TO END NOW ...,"07 23, 2014",A17YHECC8H9NEY,Margie,VERY GOOD BOOK,1406073600
982616,982616,B00M13FNSS,"[1, 1]",5,Great book! Cherika was a fool. She let that m...,"07 23, 2014",A20KO0BPMNREJL,Nicki,Great Read,1406073600
982617,982617,B00M13FNSS,"[0, 0]",5,When I say this was an excellent book please b...,"07 23, 2014",A1BQO66R6OLCCW,Nikey,Wow!!,1406073600


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982619 entries, 0 to 982618
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      982619 non-null  int64 
 1   asin            982619 non-null  object
 2   helpful         982619 non-null  object
 3   overall         982619 non-null  int64 
 4   reviewText      982597 non-null  object
 5   reviewTime      982619 non-null  object
 6   reviewerID      982619 non-null  object
 7   reviewerName    978797 non-null  object
 8   summary         982500 non-null  object
 9   unixReviewTime  982619 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 75.0+ MB


In [33]:
df.dropna(subset=['reviewText'],axis=0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 982597 entries, 0 to 982618
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      982597 non-null  int64 
 1   asin            982597 non-null  object
 2   helpful         982597 non-null  object
 3   overall         982597 non-null  int64 
 4   reviewText      982597 non-null  object
 5   reviewTime      982597 non-null  object
 6   reviewerID      982597 non-null  object
 7   reviewerName    978775 non-null  object
 8   summary         982478 non-null  object
 9   unixReviewTime  982597 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 82.5+ MB


In [36]:
df['overall'].value_counts()

overall
5    575246
4    254010
3     96193
2     34130
1     23018
Name: count, dtype: int64

In [37]:
df['overall'].unique()

array([5, 4, 3, 2, 1])

In [38]:
df_sampled=pd.DataFrame(columns=df.columns)
for category in df['overall'].unique():
    df_temp=df[df['overall']==category].sample(23018)
    df_sampled=pd.concat([df_sampled,df_temp],axis=0)
df_sampled

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
770387,770387,B00HHGX840,"[0, 0]",5,"IT WOULD HELP ALOT TO READ BOOKS 1,2 & 3. BOO...","02 13, 2014",A28N4RSZME60,Janet E Compton,GREAT SERIES,1392249600
703967,703967,B00GBGVPQU,"[0, 0]",5,This is yet another excellent story by Heather...,"11 11, 2013",A1AXY5YMMSHA5B,Amazon Customer,Third Kiss,1384128000
637314,637314,B00EY3CZJ8,"[0, 0]",5,I literally inhaled the second installment of ...,"09 7, 2013",A3GSSX9XHONVCD,Julia Busch,I'm Ready for the Next Installment,1378512000
270299,270299,B008QPCII8,"[1, 1]",5,"Where to begin?First, being someone who has al...","10 16, 2012",A1RA5OT623JO6O,"JMAuthor ""Lost in Words""",Still blushing...,1350345600
599273,599273,B00E7IWEFU,"[1, 2]",5,Amazing author!! Amazing book!! You can read t...,"08 1, 2013",A23CZR5EY7H355,TINA PHELPS,WOW!!!!!,1375315200
...,...,...,...,...,...,...,...,...,...,...
3083,3083,B001T4YZ16,"[0, 0]",1,I should have stopped reading when I read Keep...,"02 1, 2014",A89L8O00O4XFH,CJ,too much,1391212800
141446,141446,B006FOIO5S,"[26, 29]",1,The Fall of the Rogue is a novella...all sex.....,"12 28, 2011",A2W9MY06VQ3NO5,ladybug,Don't be fooled by the length of the file,1325030400
545848,545848,B00D89OGDC,"[1, 1]",1,"i dont like these types of stories, it makes t...","03 5, 2014",AHRSZYWX1DDOQ,Thismomentisfate,Girl chases guys...,1393977600
639330,639330,B00EZGN2VO,"[108, 122]",1,"Take away the sex and violence, and this novel...","09 20, 2013",A1D2C0WDCSHUWZ,"E. A Solinas ""ea_solinas""",Barbecue blues,1379635200


In [39]:
models={
    'logistic_regression':{
        'model':LogisticRegression(),
        'params':{}
    },
    'decision_tree':{
        'model':DecisionTreeClassifier(),
        'params':{}
    },
    'random_forest':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[50,100,150,200,250]
        }
    },
    'support_vector_machine':{
        'model':SVC(),
        'params':{}
    },
    'k_nearest_neighbors':{
        'model':KNeighborsClassifier(),
        'params':{
           'n_neighbors':[1,3,5,10,20,50] 
        }
    },
    'naive_bayes':{
        'model':MultinomialNB(),
        'params':{}
    }
}

In [40]:
scores={}
for k,v in models.items():
    pipln=Pipeline([
        ('tf_idf',TfidfVectorizer(ngram_range=(1,3))),
        ('classifier',v['model'])
    ])
    gscv=GridSearchCV(estimator=pipln,param_grid=v['params'],\
                      scoring='f1',cv=10)
    gscv.fit(df_sampled['reviewText'],df_sampled['overall'])
    scores[k]={
        'score':gscv.best_score_,
        'params':gscv.best_params_
    }
scores

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/siva/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/siva/.local/lib/python3.10/site-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/siva/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1204, in fit
    check_classification_targets(y)
  File "/home/siva/.local/lib/python3.10/site-packages/sklearn/utils/multiclass.py", line 218, in check_classification_targets
    raise ValueError("Unknown label type: %r" 